In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import pandas as pd
import numpy as np
import evaluate
import pickle
%load_ext autoreload
%autoreload 2

data_path = "/data/desponds/data/Summarization/"

In [ ]:
from preprocessing import preprocessing_summarization
datasets, tokenized = preprocessing_summarization()

In [ ]:
from training import get_trainers_summarization
trainers = get_trainers_summarization(data_path, tokenized)

In [ ]:
# trainers['fr'].train()
# trainers['en'].train()

## Translation

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import pandas as pd
%load_ext autoreload
%autoreload 2

data_path = "/data/desponds/data/Summarization/"

In [ ]:
import pickle
with open('/data/desponds/data/Summarization/tokenized.pickle', 'rb') as handle:
    tokenized = pickle.load(handle)
with open('/data/desponds/data/Summarization/datasets.pickle', 'rb') as handle:
    my_datasets = pickle.load(handle)
with open(f'/data/desponds/data/Summarization/translated_dataset_tokenized.pickle', 'rb') as handle:
    tokenized_fr_en = pickle.load(handle)

In [ ]:
for i in range(4, len(datasets['fr']['test'])//500 +1):
    a = 500 *i
    b = min(500*(i+1), len(datasets['fr']['test']))
    selection = list(range(a,b))
    translated_dataset = datasets['fr']['test'].select(selection).map(translate_fr_en_summarization)
    with open(f'/data/desponds/data/Summarization/translated_dataset_{a}_{b}.pickle', 'wb') as handle:
        pickle.dump(translated_dataset, handle)

In [ ]:
import datasets
all_datasets = [] 
for i in range(0, len(my_datasets['fr']['test'])//500 +1):
    a = 500 *i
    b = min(500*(i+1), len(my_datasets['fr']['test']))
    with open(f'/data/desponds/data/Summarization/translated_dataset_{a}_{b}.pickle', 'rb') as handle:
        all_datasets.append(pickle.load(handle))
translated_fr_en = datasets.concatenate_datasets(all_datasets)
# translated_fr_en = translated_fr_en.filter(lambda ex : ex['source'] != 'NoTranslation')

In [ ]:
# indices_to_remove = translated_fr_en.map(lambda ex, idx : {'idx' : idx} if ex['source'] == 'NoTranslation' else {'idx' : -100}, with_indices = True)\
#                         .filter(lambda ex : ex['idx'] != -100)['idx']
indices_to_remove = [2288, 2316, 2333, 3439, 4615, 6045, 6530, 6610]

In [ ]:
from preprocessing import preprocess_function_summarization
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
tokenized_fr_en = translated_fr_en.map(lambda examples : preprocess_function_summarization(examples, tokenizer) , batched=True)
with open(f'/data/desponds/data/Summarization/translated_dataset_tokenized.pickle', 'wb') as handle:
    pickle.dump(tokenized_fr_en, handle)

## Inference

In [ ]:
from training import get_models_summarization
models = get_models_summarization(trainers)

In [ ]:
results = {}
# results['fr'] = models['fr'].predict(tokenized['fr']['test'])
results['fr_baseline'] = models['en'].predict(tokenized['fr']['test'])
# results['en'] = models['en'].predict(tokenized['en']['test'])
# results['fr_en'] = models['en'].predict(tokenized_fr_en)
results

In [ ]:
from translation import translate_en_fr
import numpy as np
from tqdm.auto import tqdm
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base", cache_dir="/data/desponds/.cache")
preds_fr_baseline = []
for i, pred in tqdm(enumerate(results['fr_baseline'][0][:])) :
    #remove -100
    pred = np.where(pred != -100, pred, tokenizer.pad_token_id)
    #make it text
    pred = tokenizer.decode(pred, skip_special_tokens=True) 
    preds_fr_baseline.append(translate_en_fr(pred))

In [ ]:
results['fr_baseline'][0]

In [ ]:
# with open(f'/data/desponds/data/Summarization/translated_preds_en_fr.pickle', 'wb') as handle:
#     pickle.dump(translated_preds_en_fr, handle)

with open(f'/data/desponds/data/Summarization/translated_preds_en_fr.pickle', 'rb') as handle:
    translated_preds_en_fr = pickle.load(handle)

In [ ]:
import pandas as pd
final_results = {
'fr': {'test_loss': 2.488802194595337,'test_rouge1': 0.14,'test_rouge2': 0.0482,'test_rougeL': 0.1284,'test_rougeLsum': 0.1284,'test_gen_len': 18.5642,'test_runtime': 727.3367,'test_samples_per_second': 11.941,'test_steps_per_second': 1.493},
'en': {'test_loss': 2.4726946353912354,'test_rouge1': 0.2258,'test_rouge2': 0.0747,'test_rougeL': 0.1972,'test_rougeLsum': 0.1972,'test_gen_len': 18.3344,'test_runtime': 1552.1017,'test_samples_per_second': 12.308,'test_steps_per_second': 1.539},
'fr_baseline': {'test_loss': 2.4873387813568115,'test_rouge1': 0.1373,'test_rouge2': 0.0469,'test_rougeL': 0.1259,'test_rougeLsum': 0.1258,'test_gen_len': 18.6025,'test_runtime': 776.9134,'test_samples_per_second': 11.179,'test_steps_per_second': 1.398},
'fr_en': {'test_loss': 6.044681549072266,'test_rouge1': 0.0609,'test_rouge2': 0.0152,'test_rougeL': 0.0539,'test_rougeLsum': 0.0539,'test_gen_len': 18.4006,'test_runtime': 657.3901,'test_samples_per_second': 13.199,'test_steps_per_second': 1.65}}
pd.DataFrame(final_results)

## Putting all together in a df

In [ ]:
import pandas as pd
indices_to_remove = [2288, 2316, 2333, 3439, 4615, 6045, 6530, 6610]
df = {}
df['fr'] = pd.DataFrame()
df['fr']['labels']= [pred for i, pred in enumerate(my_datasets['fr']['test']['target']) if i not in indices_to_remove]
df['fr']['preds_fr'] = [pred for i, pred in enumerate(preds_fr) if i not in indices_to_remove]
df['fr']['preds_fr_en'] = translated_preds_en_fr
df['fr']['preds_fr_baseline'] = [pred for i, pred in enumerate([ p[0] for p in preds_fr_baseline]) if i not in indices_to_remove]

df['en'] = pd.DataFrame()
df['en']['labels'] = my_datasets['en']['test']['target']
df['en']['preds_en'] = preds_en

with open(f'/data/desponds/data/Summarization/all_preds.pickle', 'wb') as handle:
    pickle.dump(df, handle)

In [ ]:
import pandas as pd
import pickle
with open(f'/data/desponds/data/Summarization/all_preds.pickle', 'rb') as handle:
    df = pickle.load(handle)

# BERTscore

In [ ]:
from evaluate import load
bertscore = load("bertscore", cache_dir="/data/desponds/.cache")

In [ ]:
results_bs = {}
results_bs['fr'] = bertscore.compute(predictions=df['fr']['preds_fr'], 
                            references=df['fr']['labels'], lang="fr", rescale_with_baseline = True)

results_bs['fr_en'] = bertscore.compute(predictions=df['fr']['preds_fr_en'], 
                            references=df['fr']['labels'], lang="fr", rescale_with_baseline = True)

results_bs['en'] = bertscore.compute(predictions=df['en']['preds_en'], 
                            references=df['en']['labels'], lang="en", rescale_with_baseline = True)

results_bs['fr_baseline'] = bertscore.compute(predictions=df['fr']['preds_fr_baseline'], 
                            references=df['fr']['labels'], lang="fr", rescale_with_baseline = True)

In [ ]:
with open(f'/data/desponds/data/Summarization/bert_scores.pickle', 'wb') as handle:
    pickle.dump(results_bs, handle)

In [ ]:
with open(f'/data/desponds/data/Summarization/bert_scores.pickle', 'rb') as handle:
    handle.load(results_bs)

In [ ]:
import numpy as np
np.mean(results_bs['fr']['f1']), np.mean(results_bs['fr_en']['f1']), np.mean(results_bs['en']['f1']), np.mean(results_bs['fr_baseline']['f1'])
#(0.14974178712124486, 0.22737669299686694, 0.27177066745908673, 0.16071509587760247)

In [ ]:
df['fr']['BERTscore_fr'] = results_bs['fr']['f1']
df['fr']['BERTscore_fr_en'] = results_bs['fr_en']['f1']

df['fr']['BERTscore_fr_baseline'] = results_bs['fr_baseline']['f1']

df['en']['BERTscore_en'] = results_bs['en']['f1'] 

## Individual ROUGE scores


In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
result_fr = rouge.compute(predictions=df['fr']['preds_fr'], 
                            references=df['fr']['labels'], 
                          use_aggregator =False, 
                          rouge_types = ['rouge1', 'rouge2', 'rougeL'])
result_fr_en = rouge.compute(predictions=df['fr']['preds_fr_en'], 
                            references=df['fr']['labels'], 
                          use_aggregator =False, 
                          rouge_types = ['rouge1', 'rouge2', 'rougeL'])

result_fr_baseline = rouge.compute(predictions=df['fr']['preds_fr_baseline'], 
                            references=df['fr']['labels'], 
                          use_aggregator =False, 
                          rouge_types = ['rouge1', 'rouge2', 'rougeL'])

In [ ]:
for rouge_type in   ['rouge1', 'rouge2', 'rougeL']:
    df['fr'][f'{rouge_type}_fr'] = result_fr[rouge_type]
    df['fr'][f'{rouge_type}_fr_en'] = result_fr_en[rouge_type]
    df['fr'][f'{rouge_type}_fr_baseline'] = result_fr_baseline[rouge_type]

In [ ]:
df['fr'][f'rouge1_fr_en'].mean(), df['fr'][f'rouge2_fr_en'].mean(), df['fr'][f'rougeL_fr_en'].mean()

## Analysis BERTscore and ROUGE

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
## Check where ther is a big difference in BERTscore
df['fr'][np.abs(df['fr']['BERTscore_fr'] - df['fr']['BERTscore_fr_en'])>0.6 ]

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1,3, figsize = (14,4))
alpha = 0.3
axs = axs.ravel()
df['fr']['BERTscore_fr'].plot(kind= 'hist', alpha = alpha, title = "Distribution of BERTscore", ax = axs[0], label ="$metric_{fr}$", bins = 20)
df['fr']['BERTscore_fr_en'].plot(kind= 'hist', alpha = alpha,  ax = axs[0], label ="$metric_{fr-en}$", bins = 20)
df['fr']['BERTscore_fr_baseline'].plot(kind= 'hist', alpha = alpha,  ax = axs[0], label ="$metric_{fr-baseline}$", bins = 20)
axs[0].legend()

# df['fr']['rouge1_fr'].hist(alpha = 0.5)
# df['fr']['rouge1_fr_en'].hist(alpha = 0.5)
df['fr']['rouge1_fr'].plot(kind= 'hist', alpha = alpha, title = "Distribution of ROUGE-1 score", ax = axs[1], label ="$metric_{fr}$", bins = 20)
df['fr']['rouge1_fr_en'].plot(kind= 'hist', alpha = alpha,  ax = axs[1], label ="$metric_{fr-en}$", bins = 20)
df['fr']['rouge1_fr_baseline'].plot(kind= 'hist', alpha = alpha,  ax = axs[1], label ="$metric_{fr-baseline}$", bins = 20)
axs[1].legend()

df['fr']['rouge2_fr'].plot(kind= 'hist', alpha = alpha, title = "Distribution of ROUGE-2 score", ax = axs[2], label ="$metric_{fr}$", bins = 20)
df['fr']['rouge2_fr_en'].plot(kind= 'hist', alpha = alpha,  ax = axs[2], label ="$metric_{fr-en}$", bins = 20)
df['fr']['rouge2_fr_baseline'].plot(kind= 'hist', alpha = alpha,  ax = axs[2], label ="$metric_{fr-baseline}$", bins = 20)
axs[2].legend()

In [ ]:
df['fr'][np.abs(df['fr']['rouge1_fr'] - df['fr']['rouge1_fr_en'])>0.4 ][['labels', 'preds_fr', 'preds_fr_en',
       'rouge1_fr', 'rouge1_fr_en']]

In [ ]:
df['fr'][np.abs(df['fr']['rouge1_fr'] - df['fr']['rouge1_fr_en'])>0.4 ][['labels', 'preds_fr', 'preds_fr_en',
       'rouge2_fr', 'rouge2_fr_en']]

In [ ]:
diff_rouge, diff_bert = 0.1,0.2
df['fr'][((df['fr']['rouge1_fr'] - df['fr']['rouge1_fr_en']>diff_rouge) 
          & (df['fr']['BERTscore_fr'] - df['fr']['BERTscore_fr_en']< - diff_bert)& (df['fr']['BERTscore_fr'] >0.2)) 
         | ((df['fr']['rouge1_fr'] - df['fr']['rouge1_fr_en']< -diff_rouge) 
          & (df['fr']['BERTscore_fr'] - df['fr']['BERTscore_fr_en'] > diff_bert)& (df['fr']['BERTscore_fr'] >0.2)) ][['labels', 'preds_fr', 'preds_fr_en',
       'BERTscore_fr', 'BERTscore_fr_en','rouge1_fr', 'rouge1_fr_en']]

In [ ]:
diff_rouge, diff_bert = 0.1,0.1
df['fr'][((df['fr']['rouge2_fr'] - df['fr']['rouge2_fr_en']>diff_rouge) 
          & (df['fr']['BERTscore_fr'] - df['fr']['BERTscore_fr_en']< - diff_bert)& df['fr']['BERTscore_fr_en'] >0.4)
         | ((df['fr']['rouge2_fr'] - df['fr']['rouge2_fr_en']< -diff_rouge) 
          & (df['fr']['BERTscore_fr'] - df['fr']['BERTscore_fr_en'] > diff_bert)& df['fr']['BERTscore_fr_en'] >0.4)][['labels', 'preds_fr', 'preds_fr_en',
       'BERTscore_fr', 'BERTscore_fr_en','rouge2_fr', 'rouge2_fr_en']].loc[[1302, 8502, 8098, 2284, 100]]
#Interesting sample_ids 100 8502 8098 2284 1302

In [ ]:
diff_rouge, diff_bert = 0.1,0.2
df['fr'][((df['fr']['rougeL_fr'] - df['fr']['rougeL_fr_en']>diff_rouge) 
          & (df['fr']['BERTscore_fr'] - df['fr']['BERTscore_fr_en']< - diff_bert)) 
         | ((df['fr']['rougeL_fr'] - df['fr']['rougeL_fr_en']< -diff_rouge) 
          & (df['fr']['BERTscore_fr'] - df['fr']['BERTscore_fr_en'] > diff_bert)) ][['labels', 'preds_fr', 'preds_fr_en',
       'BERTscore_fr', 'BERTscore_fr_en','rougeL_fr', 'rougeL_fr_en']]
#Interesting sample_ids 2 100 573 
# Downside of ROUGE 
# ne prend pas en compte les pluriels
# Si suite de petit mot comme Quand on doit, va augmenter le score alors que ça ne donne aucune information

# Downside of BERTscore
# 

In [ ]:
df['fr'][['labels', 'preds_fr', 'preds_fr_en',
       'BERTscore_fr', 'BERTscore_fr_en','rouge1_fr', 'rouge1_fr_en']].loc[[1302, 8502, 8098, 2284, 100]]

In [ ]:
#The number of times berscores and rouge agree
len(df['fr'][((df['fr']['rouge1_fr'] > df['fr']['rouge1_fr_en']) & (df['fr']['BERTscore_fr'] > df['fr']['BERTscore_fr_en']))|
            ((df['fr']['rouge1_fr'] < df['fr']['rouge1_fr_en']) & (df['fr']['BERTscore_fr'] < df['fr']['BERTscore_fr_en']))
            ][['labels', 'preds_fr', 'preds_fr_en',
       'BERTscore_fr', 'BERTscore_fr_en','rougeL_fr', 'rougeL_fr_en']]), len(df['fr'])

In [ ]:
#The number of times berscores and rouge agree
len(df['fr'][((df['fr']['rouge2_fr'] > df['fr']['rouge2_fr_en']) & (df['fr']['BERTscore_fr'] > df['fr']['BERTscore_fr_en']))|
            ((df['fr']['rouge2_fr'] < df['fr']['rouge2_fr_en']) & (df['fr']['BERTscore_fr'] < df['fr']['BERTscore_fr_en']))
            ][['labels', 'preds_fr', 'preds_fr_en',
       'BERTscore_fr', 'BERTscore_fr_en','rougeL_fr', 'rougeL_fr_en']]), len(df['fr'])

In [ ]:
#The number of times berscores and rouge agree
len(df['fr'][((df['fr']['rougeL_fr'] > df['fr']['rougeL_fr_en']) & (df['fr']['BERTscore_fr'] > df['fr']['BERTscore_fr_en']))|
            ((df['fr']['rougeL_fr'] < df['fr']['rougeL_fr_en']) & (df['fr']['BERTscore_fr'] < df['fr']['BERTscore_fr_en']))
            ][['labels', 'preds_fr', 'preds_fr_en',
       'BERTscore_fr', 'BERTscore_fr_en','rougeL_fr', 'rougeL_fr_en']]), len(df['fr'])

In [ ]:
df['fr']['preds_fr_baseline']

# Results

In [ ]:
import pandas as pd
data = {
    'task' : ['Summarizing','Summarizing', 'Summarizing', 'Summarizing'],
    'model'       : ['RoBERTa', 'RoBERTa', 'RoBERTa', 'CamemBERT'],
    'train_dataset' : ['GEM/wikilingua en', 'GEM/wikilingua en', 'GEM/wikilingua en', 'GEM/wikilingua fr'],
    'nb_sample_train' : [87599, 87599,  87599, 20731],
    'test_dataset' : ['GEM/wikilingua en', 'GEM/wikilingua fr', 'GEM/wikilingua fr translated', 'GEM/wikilingua fr',],
    'translated' : ['no', 'no', 'yes', 'no'],
    'ROUGE 1'    : [1,1,1,1],
    'BERTscore'   : [0.27177, 0.16071,  0.22737, 0.14974]
}
results = pd.DataFrame(data)
results